In [2]:
from pathlib import Path

PATH_DIR_DATA = "/home/user/karacora/data"
NAME_DATASET = "ScanNet"
NAME_SCENE = "scene0000_00"

path_dataset = Path(PATH_DIR_DATA) / NAME_DATASET

In [4]:
import json

import numpy as np
import torch
import open3d as o3d


def convert_pc_to_box(obj_pc):
    xmin = np.min(obj_pc[:, 0])
    ymin = np.min(obj_pc[:, 1])
    zmin = np.min(obj_pc[:, 2])
    xmax = np.max(obj_pc[:, 0])
    ymax = np.max(obj_pc[:, 1])
    zmax = np.max(obj_pc[:, 2])
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2, (zmin + zmax) / 2]
    box_size = [xmax - xmin, ymax - ymin, zmax - zmin]
    return center, box_size


def load_scan(pcd_path, inst2label_path, scene_name):
    pcd_data = torch.load(pcd_path / f"{scene_name}.pth", weights_only=False)
    inst_to_label = torch.load(inst2label_path / f"{scene_name}.pth", weights_only=False)
    points, colors, instance_labels = pcd_data[0], pcd_data[1], pcd_data[-1]
    pcds = np.concatenate([points, colors], 1)
    return points, colors, pcds, instance_labels, inst_to_label


def visualize_one_scene(obj_pcds, points, colors, caption):
    # visualize scene
    o3d_pcd = o3d.geometry.PointCloud()
    o3d_pcd.points = o3d.utility.Vector3dVector(points)
    o3d_pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)
    # visualize gt
    boxes = []
    labels = []
    for idx, (obj, obj_label) in enumerate(obj_pcds):
        gt_center, gt_size = convert_pc_to_box(obj)
        gt_o3d_box = o3d.geometry.OrientedBoundingBox(gt_center, np.eye(3, 3), gt_size)
        gt_o3d_box.color = [0, 1, 0]
        boxes.append(gt_o3d_box)
        labels.append(obj_label)

    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[-0, -0, -0])
    print(labels)
    o3d.visualization.draw_geometries([o3d_pcd, *boxes, mesh_frame], window_name=caption)


def visualize_data(save_root, scene_name, vis_obj=True):
    inst2label_path = save_root / "instance_id_to_label"
    pcd_path = save_root / "pcd_with_global_alignment"

    points, colors, pcds, instance_labels, inst_to_label = load_scan(pcd_path, inst2label_path, scene_name)

    if not vis_obj:
        o3d_pcd = o3d.geometry.PointCloud()
        mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6, origin=[-0, -0, -0])
        o3d_pcd.points = o3d.utility.Vector3dVector(points)
        o3d_pcd.colors = o3d.utility.Vector3dVector(colors / 255.0)
        o3d.visualization.draw_geometries([mesh_frame, o3d_pcd])
        return

    obj_pcds = []
    for i in inst_to_label.keys():
        mask = instance_labels == i  # time consuming
        if np.sum(mask) == 0:
            continue
        obj_pcds.append((pcds[mask], inst_to_label[i]))

    visualize_one_scene(obj_pcds, points, colors, scene_name)


def visualize_refer(save_root, anno_file):
    inst2label_path = save_root / "instance_id_to_label"
    pcd_path = save_root / "pcd_with_global_alignment"
    json_data = json.load(open(anno_file, "r", encoding="utf8"))
    for item in json_data:
        scan_id = item["scan_id"]
        inst2label_path = save_root / "instance_id_to_label"
        pcd_path = save_root / "pcd_with_global_alignment"

        inst_to_label = torch.load(inst2label_path / f"{scan_id}.pth", weights_only=False)
        pcd_data = torch.load(pcd_path / f"{scan_id}.pth", weights_only=False)
        points, colors, instance_labels = pcd_data[0], pcd_data[1], pcd_data[-1]
        pcds = np.concatenate([points, colors], 1)

        target_id = int(item["target_id"])
        mask = instance_labels == target_id
        if np.sum(mask) == 0:
            continue

        obj_pcds = [(pcds[mask], inst_to_label[target_id])]
        visualize_one_scene(obj_pcds, points, colors, scan_id + "-" + item["utterance"])

In [7]:
visualize_data(path_dataset / "scan_data", scene_name=NAME_SCENE)

['window', 'window', 'table', 'kitchen counter', 'shower', 'curtain', 'curtain', 'desk', 'cabinet', 'floor', 'sink', 'scale', 'trash can', 'trash can', 'trash can', 'tv', 'pillow', 'clock', 'backpack', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'wall', 'stool', 'stool', 'stool', 'stool', 'couch', 'refrigerator', 'coffee table', 'table', 'toilet', 'bed', 'cabinet', 'cabinet', 'kitchen cabinets', 'cabinet', 'cabinet', 'kitchen cabinets', 'object', 'toaster oven', 'laundry basket', 'guitar', 'tissue box', 'nightstand', 'nightstand', 'dish rack', 'microwave', 'toaster', 'door', 'ceiling', 'shelf', 'bicycle', 'shoes', 'object', 'object', 'object', 'object', 'doorframe', 'mirror', 'doorframe', 'guitar case']


error: XDG_RUNTIME_DIR not set in the environment.


KeyboardInterrupt: 

In [ ]:
visualize_refer(path_dataset / "scan_data", anno_file=path_dataset / "annotations/refer" / "")